[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shimizu-biz/nlp_workshop/blob/master/step2.ipynb)

# step 2

不要な警告を非表示にする

In [0]:
import warnings
warnings.filterwarnings('ignore')

gensimのインストールを行いましょう
gensimは自然言語処理のためにしばしば使われる（ベクトル空間モデル、トピックモデル）ライブラリです

In [55]:
%%bash
pip3 install gensim

gensimのインポート

In [0]:
import gensim
from gensim.models import word2vec

本来は学習のプロセスも体験していただきたいのですが、学習にはかなり時間がかかるので、有志の方の学習済みもモデルを使わせてもらいます

In [57]:
%%bash
curl -L -o pretrained_model.bin.zip  "http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip"
unzip pretrained_model.bin.zip

Archive:  pretrained_model.bin.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  126M  100  126M    0     0  4725k      0  0:00:27  0:00:27 --:--:-- 15.8M
replace word2vec.gensim.model? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


モデルのロード

In [0]:
model = word2vec.Word2Vec.load('word2vec.gensim.model')

語彙数の確認しましょう

In [59]:
len(model.wv.vocab)

335476

語彙（一部）を表示しましょう

In [60]:
model.wv.vocab

{'『': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a4b1d3f60>,
 'GO': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a3f77ddd8>,
 '!': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a3f77de48>,
 'ガリバー': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a3f77dbe0>,
 'くん': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a3f77db38>,
 '』': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a3f77dba8>,
 '（': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a3f77dc50>,
 'ゴーゴー': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a3f77de80>,
 '）': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a3f77df60>,
 'は': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a3f77dd30>,
 '、': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a4d77ff60>,
 '関西テレビ': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a571b4048>,
 'ほか': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a571b4080>,
 'で': <gensim.models.deprecated.keyedvectors.Vocab at 0x7f6a571b

東京にどのようなベクトルが割り当てられているか確認しましょう

In [61]:
model.wv['東京']

array([ 1.46078676e-01,  1.45535290e-01,  5.83377965e-02, -1.29224882e-01,
        7.21001700e-02,  1.29963398e-01, -6.78467527e-02, -1.21103026e-01,
       -1.80525705e-02,  2.32675970e-01,  5.17494902e-02, -7.52029046e-02,
       -1.22526415e-01,  1.30010486e-01, -1.42008409e-01, -1.49845868e-01,
       -1.48141682e-01,  4.41302136e-02,  1.42423093e-01, -1.18190199e-02,
       -8.33276734e-02, -1.71803966e-01,  3.05940896e-01,  1.97436358e-03,
       -2.33649671e-01,  3.43715400e-02, -1.85629278e-01,  4.88921739e-02,
        2.48211861e-01,  2.92468458e-01, -1.12550400e-01,  1.10607676e-01,
        2.60777563e-01, -5.09645753e-02,  7.65050427e-05, -7.01758116e-02,
        1.37903079e-01, -3.64999950e-01, -3.01632676e-02, -1.37715772e-01,
       -5.45540415e-02,  4.47592624e-02, -7.82924592e-02,  5.02642319e-02,
       -1.84998602e-01,  2.47902684e-02, -4.89305928e-02,  1.20796755e-01,
       -1.47180483e-02, -7.94531405e-02], dtype=float32)

コサイン類似度に基づいて、語彙の中で最も「東京」に近い単語を表示しましょう

In [62]:
model.wv.most_similar(positive=["東京"], topn=1)

[('大阪', 0.9429634809494019)]

Question:「大きい」に最も近い単語は何でしょうか？

1. 大きな
2. 大
3. 大きさ
3. 小さい

In [63]:
model.wv.most_similar(positive=["大きい"], topn=1)

[('小さい', 0.9054060578346252)]

他の候補はどうだったのでしょうか？

In [64]:
print (model.wv.similarity('大きい','大きな'))
print (model.wv.similarity('大きい','大'))
print (model.wv.similarity('大きい','大きな'))
print (model.wv.similarity('大きい','小さい'))

0.5659633
0.2546202
0.5659633
0.90540606


ベクトルなので足し算・引き算ができます。
日本にとっての東京は、アメリカにとっての何でしょうか？

In [65]:
vector = model.wv['東京'] - model.wv['日本'] + model.wv['アメリカ']
model.wv.similar_by_vector(vector, topn=1, restrict_vocab=None)

[('ニューヨーク', 0.7787365913391113)]

文のベクトルを表現する簡単な方法の一つとして、単語のベクトルの和を考えることができます。
これにより、文同士のコサイン類似度を計算することもできます。

In [0]:
import MeCab
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
def mas_ls(text):#mecabを使って分かち書き
    m = MeCab.Tagger("-Owakati")
    return m.parse(text)[:-1]
def word_vector_sum(ls):#単語ベクトルの和を計算
    vec = np.zeros(model.wv.vector_size)
    for word in ls:
        try:
            vec += model.wv[word]
        except:
            pass
    return vec
def calc_sim(s1,s2):
    v1 = word_vector_sum(mas_ls(s1))
    v2 = word_vector_sum(mas_ls(s2))
    return cosine_similarity(v1.reshape(1,-1),v2.reshape(1,-1))#ベクトルのコサイン類似度を計算

文のペアが二つあります。どちらのペアが似ている組み合わせだと思いますか。

In [67]:
s1 = "今日は晴れだ"
s2 = "本日は快晴だ"
print (calc_sim(s1,s2))

s1 = "今日は晴れだ"
s2 = "好きな２文を入れてみましょう"
print (calc_sim(s1,s2))

[[0.70166144]]
[[0.3785033]]


この方法も、語順の情報が失われてしまいます

In [68]:
s1 = "りんごは好きだが、ぶどうは嫌いだ"
s2 = "ぶどうは好きだが、りんごは嫌いだ"
print (calc_sim(s1,s2))

[[1.]]
